<a href="https://colab.research.google.com/github/ArminVarmaz/Schufa/blob/main/LogRegression_Schufa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Untersuchung von SCHUFA - Score-Simulators

Diese Datei zeigt die Schritte der Analyse auf und soll die Kommunikation vereinfachen. 


Es handelt sich um ein Python-Code. Um die Untersuchungen machen zu können, benötigt Python mehrere Bibliotheken, die nachfolgend geladen werden.



In [16]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tools import add_constant

Erste Daten einlesen. Die Daten sind auf einem privaten Github, zu dem ich Euch eingeladen habe.

In [17]:
# Daten einlesen
url='https://github.com/ArminVarmaz/Schufa/blob/main/Data/Choices_komplett.xlsx?raw=true'
df = pd.read_excel(url, sheet_name="Alle_Permutationen")

/usr/local/lib/python3.9/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Bevor die eigentliche Analyse gemacht werden kann, müssen die Daten umkodiert werden.

Ich benutze später die logistische Version, für die wir die Ergebnisse (in unserem Fall die EInstufung durch Schufa) in eine 1 und 0 kodieren müssen. Im Code, der hier nachfolgt, werden die Einstufungen 'Ausreichend' und 'Ungenügend' als 1 festgelegt. Die Ergebnisse der Regression sind daher in diesem Sinne zu interpretieren. Die Interpretation ist dann: 
* Variablen mit positive Koeffizienten erhöhen die Wahrscheinlichkeit des Ausfalls
* Variablen mit negativen Koeffizienten senken die Wahrscheinlichkeit des Ausfalls

Im Code könnt Ihr die Einstufung ändern, dann erhalten wir auch andere Ergebnisse.


In [18]:
# Ergebnis ersetzenr
replace_dict = {'Hervorragend': 0, 'gut': 0, 'Akzeptabel': 0, 'Ausreichend': 1, 'Ungenügend': 1}
df['Ergebnis'] = df['Ergebnis'].replace(replace_dict)

# Spaltennamen für die neuen Variablen generieren
new_cols = []
for col in df.columns[1:7]:
    for ans in sorted(df[col].unique()):
        new_cols.append(col + "_" + ans)

# Neue Variablen generieren und an das DataFrame anhängen
for col in df.columns[1:7]:
    for ans in sorted(df[col].unique()):
        df[col + "_" + ans] = np.where(df[col] == ans, 1, 0)

Ferer sind auch alle X-Variablen als Dummy-Variablen kodiert. Bei Dummy-Variablen, die wir als X-Variablen nutzen, muss immer eine Basisgruppe definiert werden und alle Ergebnisse sind in Bezug zur jeweiligen Basisgruppe zu interpretieren. Die Basisgruppe wird aus der eigentlichen empirischen Schätzung entfernt. Das sind die Variablen, die hier mit X.drop weggelassen werden. Je nachdem, was die Basisvariable ist, wird die Interpretation der Ergebnisse leicht anders. Zur Interpretation komme ich später. 

In [19]:
# Modell auf Basis der neuen Variablen erstellen
X = add_constant(df[new_cols])
X = X.drop(["Kreditkarten_'Keine Kreditkarten'", "Ratenkredite_'Keine'", "Immobilienkredite_'Nein'",
            "Online-Käufe auf Rechnung_'0 bis 5 Mal'", "Umzug_'Vor weniger als 3 Jahren'", "Zahlungsausfälle_'Ja, aber ich habe schon bezahlt'"],
           axis=1)
y = df['Ergebnis']

Logistische Regression durchführen und Ergebnisse anzeigen:

In [20]:
# Logistische Regression mit Statsmodels
logit_model = sm.Logit(y, X)
result = logit_model.fit(maxiter=1000)

# Ergebnisse ausgeben
summary = result.summary2()
print(summary)

         Current function value: 0.144666
         Iterations: 1000
                                                 Results: Logit
Model:                               Logit                            Pseudo R-squared:                 0.786   
Dependent Variable:                  Ergebnis                         AIC:                              777.9486
Date:                                2023-03-29 14:01                 BIC:                              859.9912
No. Observations:                    2592                             Log-Likelihood:                   -374.97 
Df Model:                            13                               LL-Null:                          -1754.9 
Df Residuals:                        2578                             LLR p-value:                      0.0000  
Converged:                           0.0000                           Scale:                            1.0000  
No. Iterations:                      1000.0000                               

/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Die Ergebnisse der logistischen Regresssion sind im Bezug auf die Wahrscheinlickeit des Ereignisses zu interpretieren. Oben habe ich als Ereignis die Fälle 'ausreichende' und 'ungenügende' Bonität festgelegt. Schauen wir uns die Ergebnisse an:


1.   **Variable** Kreditkarten_'1-2 Kreditkarten': Der Koeffizient ist -1.3525, d.h. die Wahrscheinleichkeit des Ereignisses wird gesenkt. Also, wenn eine Person 1-2 Kreditkarte besitzt, wird die Wahrscheinkeit gesenkt, als 'ausreichende' und 'ungenügende' Bonität klassifiziet zu werden ijm Vergleich zu einer Person aus der Basisgruppe (Keine_Kreditkarten)
2.   **Variable** Zahlungsausfälle_'Nein': Der Koeffiizent ist -7.3478, d.h. die Wahrscheinleichkeit des Ereignisses wird hier auch gesenkt (im Vergleich zur Basisgruppe). 
3.   **Variable** Online-Käufe auf Rechnung_'Mehr als 5 Mal': Der Koeffizient ist 2.4372, d.h. die Wahrscheinlicht des Ereignisses wird für diese Person erhöht (im Vergleich zur Basisgruppe).




Relative Bedeutung der Variablen bestimmen. Die insignifikanten Variablen werden entfernt.

Insgesamt sind die Zahlungsausfälle sehr wichtig!

Als nächstes bestimmen wir die Odds und die relativen Anteile:


In [28]:
# Spalten aus X auswählen, die signifikant sind
sig_cols = result.pvalues[result.pvalues < 0.05].index
X_sig = X[sig_cols]

# Relative Bedeutung der signifikanten Variablen berechnen
odds = np.exp(result.params[sig_cols])
rel_importance = odds / np.exp(result.params[sig_cols]).sum()

# Formatierung der relativen Anteile
odds_formatted = odds.map(lambda x: '{:.2}'.format(x))
rel_importance_formatted = rel_importance.map(lambda x: '{:.2%}'.format(x))

print("\n\nOdds: \n\n")
print(odds_formatted)
print("\n\nRelative Anteile: \n\n")
print(rel_importance_formatted)




Odds: 


const                                         2.8e+01
Kreditkarten_'1-2 Kreditkarten'                  0.26
Ratenkredite_'2 Ratenkredite'                     2.7
Ratenkredite_'3 Ratenkredite'                     9.1
Ratenkredite_'4 Ratenkredite'                 1.8e+01
Ratenkredite_'Mehr als 4 Ratenkredite'        5.6e+01
Immobilienkredite_'Ja'                          0.055
Online-Käufe auf Rechnung_'Mehr als 5 Mal'    1.1e+01
Umzug_'Vor 3 bis 10 Jahren'                     0.072
Umzug_'Vor mehr als 10 Jahren'                  0.013
Zahlungsausfälle_'Nein'                       0.00064
dtype: object


Relative Anteile: 


const                                         22.61%
Kreditkarten_'1-2 Kreditkarten'                0.21%
Ratenkredite_'2 Ratenkredite'                  2.12%
Ratenkredite_'3 Ratenkredite'                  7.21%
Ratenkredite_'4 Ratenkredite'                 14.14%
Ratenkredite_'Mehr als 4 Ratenkredite'        44.49%
Immobilienkredite_'Ja'                  

Die Odds-Verhältnisse geben das Verhältnis der Chancen (Odds) an, dass das Ereignis in der Kategorie der einen Variable gegenüber der Kategorie der anderen Variable auftritt. Ein Odds-Verhältnis größer als 1 deutet darauf hin, dass die Chancen, dass das Ereignis in der einen Kategorie auftritt, größer sind als in der anderen Kategorie. Ein Odds-Verhältnis kleiner als 1 deutet darauf hin, dass die Chancen, dass das Ereignis in der einen Kategorie auftritt, kleiner sind als in der anderen Kategorie.

Im Beispiel bedeutet beispielsweise ein Odds-Verhältnis von 0.26 für die Variable "Kreditkarten_'1-2 Kreditkarten'", dass die Chancen, dass das Ereignis in der Kategorie "1-2 Kreditkarten" auftritt, etwa ein Viertel so groß sind wie in der Vergleichskategorie. Ein Odds-Verhältnis von 9.1 für die Variable "Ratenkredite_'3 Ratenkredite'" bedeutet, dass die Chancen, dass das Ereignis in der Kategorie "3 Ratenkredite" auftritt, etwa neunmal höher sind als in der Vergleichskategorie.

Die relativen Anteile sind die Gewichtungen der Variablen, wenn das definierte Ereignis auftritt. Also, wenn eine Person 'Mehr als 4 Ratenkredite' hat, dann trägt es schon 44.5% zum negativen Rating bei.

